<a href="https://www.kaggle.com/code/kushalpatel1124/swiggyy?scriptVersionId=142965083" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%pip install -q ipywidgets
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets , Layout, Box, Output, Dropdown, interactive

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
file = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(file[0])
df

In [ ]:
df.info()

In [ ]:
restaurents = df['City'].value_counts()

In [ ]:
plt.figure(figsize=(10, 5)) 
restaurents.plot(kind = 'bar' , color = 'blue')
plt.xlabel('Cities')
plt.ylabel('Count')
plt.title('Total Restaurents')
plt.show()

In [ ]:
Food = []
for i in df['Food type']:
    if len(i.split(",")) > 1:
        for c in i.split(","):
            Food.append(c.strip())
    elif len(i.split("  ")) > 1:
        for c in i.split("  "):
            Food.append(c.strip())
    else:
        Food.append(i.strip())

In [ ]:
food_count = pd.Series(Food)
food_count = food_count.replace(['Use code XPRESS121 to avail.','Special Discount from (The Scoop)','SVANidhi Street Food Vendor','Offer Valid only for New Customers', 'Attractive Combos Available', 'Popular Brand Store', 'indian', 'Punjab'], "N/A")
total_count = food_count.value_counts()

In [ ]:
plt.figure(figsize=(10, 20)) 
total_count.plot(kind = 'barh' , color = 'blue')
plt.xlabel('Food Type')
plt.ylabel('Count')
plt.title('Best Food Type')
plt.show()

In [ ]:
for f in food_count:
    df[f] = np.zeros(len(df), dtype=int)

In [ ]:
for f in food_count:
    df[f] = df['Food type'].apply(lambda x: 1.0 if f in x else 0.0)

In [ ]:
df['Excellent'] = df['Avg ratings'].between(4 , 5).astype(int)
df['Good'] = df['Avg ratings'].between(2,3).astype(int)

In [ ]:
food_set = set(food_count)

In [ ]:
food_sum = df[(df['City'] == "Bangalore")][list(food_set)].sum()

In [ ]:
food_sum

In [ ]:
plt.figure(figsize=(20, 5)) 
food_sum.plot(kind = 'bar' , color = 'blue')
plt.xlabel('food type')
plt.ylabel('Count')
plt.title('Best food type in Banglore')
plt.show()

In [ ]:
def create_graph(city='', restaurant=''):
    filtered_df = df.copy()
    if city:
        filtered_df = filtered_df[(filtered_df['City'] == city)]
    if restaurant != 'All':
        filtered_df = filtered_df[(filtered_df['Restaurant'] == restaurant)]
    
    if restaurant == 'All':
        food_sum = filtered_df[list(food_set)].sum().sort_values(ascending=False)
    else:
        food_sum = filtered_df[list(food_set)].sum()
    
    plt.figure(figsize=(20, 5))
    food_sum.plot(kind='bar', color='blue')
    plt.xlabel('Food type')
    plt.ylabel('Count')
    plt.title(f'Best food type in {city}')
    plt.show()

def update_restaurants_options(change):
    selected_city = change['new']
    filtered_restaurants = sorted(list(df[df['City'] == selected_city]['Restaurant'].unique()))
    restaurant_widget.options = [('All', 'All')] + [(r, r) for r in filtered_restaurants]

cit = df['City'].unique()
city_widget = widgets.Dropdown(
    options=cit,
    value='Bangalore',
    description='Cities:'
)

restaurant_widget = widgets.Dropdown(
    options=[],
    description='Restaurants:'
)
city_widget.observe(update_restaurants_options, names='value')

out = widgets.interactive_output(create_graph, {'city': city_widget, 'restaurant': restaurant_widget})

kp = widgets.VBox([city_widget, restaurant_widget, out])
kp

In [ ]:
food_sum2 = df[(df['City'] == "Bangalore") & (df['Excellent'] == 1)][list(food_set)]

In [ ]:
restaurant_names = df['Restaurant'].tolist()

In [ ]:
best_res = df[(df['Restaurant'] == restaurant_names) & (df['Excellent'] == 1) & (df['City'] == 'Surat')]

In [ ]:
best_res

In [ ]:
def best_food(city, food_type, restaurant):
    filtered_df2 = df.copy()
    if city:
        filtered_df2 = filtered_df2[filtered_df2['City'] == city]
    if food_type:
        filtered_df2 = filtered_df2[filtered_df2[list(food_set)] == food_type]
    if restaurant:
        filtered_df2 = filtered_df2[filtered_df2['Restaurant'] == restaurant]

    return filtered_df2

def updated_food_options(change):
    selected_city = change['new']
    filtered_city = df[df['City'] == selected_city]
    food_counts = filtered_city[list(food_set)].sum()
    food_counts_sorted = food_counts.sort_values(ascending=False)
    filtered_food = list(food_counts_sorted.index)

    foods_widget.options = [(food, food) for food in filtered_food]

def updated_restaurant(change):
    selected_food = change['new']
    selected_city = city_widget.value
    filtered_restaurantss = df[(df['City'] == selected_city) & (df[selected_food] == 1) & (df['Excellent'] == 1)]
    restaurant_options = filtered_restaurantss['Restaurant'].unique()

    restaurant_widget.options = [(restaurant, restaurant) for restaurant in restaurant_options]

def updated_area_options(change):
    selected_restaurant = change['new']
    selected_area = city_widget.value
    filtered_areas = df[(df['City'] == selected_area) & (df['Restaurant'] == selected_restaurant)]['Area'].unique()

    area_widget.options = [(area, area) for area in filtered_areas]

cit = df['City'].unique()

city_widget = widgets.Dropdown(
    options=cit,
    value='Surat',
    description='Cities:'
)

foods_widget = widgets.Dropdown(
    options=[],
    description='Best Food:'
)

restaurant_widget = widgets.Dropdown(
    options=[],
    description='Restaurant:'
)

area_widget = widgets.Dropdown(
    options=[],
    description='Area:'
)

city_widget.observe(updated_food_options, names='value')
foods_widget.observe(updated_restaurant, names='value')
restaurant_widget.observe(updated_area_options, names='value')

out = widgets.interactive_output(best_food, {'city': city_widget, 'food_type': foods_widget, 'restaurant': restaurant_widget})

kushal = widgets.VBox([city_widget, foods_widget, restaurant_widget, area_widget, out])
kushal

In [ ]:
from wordcloud import WordCloud

plt.subplots(figsize =(8,8))
wordcloud = WordCloud(
                            background_color = 'white',
                            width = 512,
                            height = 384
                        ).generate(" ".join(df[list(food_set)]))

plt.imshow(wordcloud)
plt.axis("off")
plt.savefig('graph.png')

plt.show()